# Kerchunk and Xarray-Datatree at Scale

## Overview

In the previous tutorials we have demonstrated how to use Kerchunk on small collections of files. In this tutorial we are going to use a large collection of pre-generated Kerchunk reference files and open them with [xarray-datatree](https://xarray-datatree.readthedocs.io/en/latest/).


### About the Dataset

This collection of reference files were generated from the [**NASA NEX-GDDP-CMIP6 (Global Daily Downscaled Projections)**](https://www.nasa.gov/nex/gddp/) dataset.  A version of this dataset is hosted on `s3` as a collection of NetCDF files. 


## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) | Required | Core |
| [Kerchunk and Dask](../foundations/kerchunk_dask) | Required | Core |
| [Multi-File Datasets with Kerchunk](../case_studies/ARG_Weather.ipynb) | Required | IO/Visualization |
| [Xarray-Datatree Overview](https://xarray-datatree.readthedocs.io/en/latest/quick-overview.html)| Required | IO |

- **Time to learn**: 30 minutes

## Motivation

In total the dataset is roughly **12TB**, with a single NetCDF file per yearly timestep, per variable. Downloading this entire dataset for analysis on a local machine would difficult to say the least. The collection of Kerchunk reference files for this entire dataset is only **272 Mb**, which is about 42,000 times smaller! 

## Imports

In [ ]:
from datatree import DataTree
import xarray as xr
import pandas as pd
import fsspec
import dask
from distributed import Client
from fsspec.implementations.reference import LazyReferenceMapper, ReferenceFileSystem

import hvplot
import hvplot.xarray

## Read the Reference Catalog

The **NASA NEX-GDDP-CMIP6** dataset is organized by GCM, Scenario and Ensemble Member. Each of these Scenario/GCM combinations is represented as a combined reference file, which was created by merging across variables and concatenating along time-steps. All of these references are organized into a simple .csv catalog in the schema: 
|   GCM_Scenario  |      url    |
| --------------- | ----------- |


# Organzing with Xarray-Datatree
Not all of the GCM/Scenario reference datasets have shared spatial coordinates and many of the have slight differences in their calendar and thus time dimension. 
Because of this, these cannot be combined into a single `Xarray-Dataset`. Fortunatly `Xarray-Datatree` provides a higher level abstraction where related `Xarray-Datasets` are organized into a tree structure where each dataset corresponds to a `leaf`.

In [ ]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_prod.csv"
)

# Convert the DataFrame into a dictionary
catalog = cat_df.set_index("ID").T.to_dict("records")[0]

## Load Reference Datasets into Xarray-DataTree

In the following cell we create a function `load_ref_ds`, which can be parallealized via Dask to load Kerchunk references into a dictionary of `Xarray-Datasets`. 

In [ ]:
def load_ref_ds(url: str):

    fs = ReferenceFileSystem(
        url,
        remote_protocol="s3",
        target_protocol="s3",
        target_options={"anon": True},
        lazy=True,
    )
    return xr.open_dataset(
        fs.get_mapper(), engine="zarr", backend_kwargs={"consolidated": False}
    )


tasks = {id: dask.delayed(load_ref_ds)(url) for id, url in catalog.items()}

## Use Dask Distributed to load the Xarray-Datasets from Kerchunk reference files
Using Dask, we're loading 164 reference datasets into memory. Since they're are Xarray datasets the coordinates are loaded eagerly, but the underlying data is still lazy. 

In [ ]:
client = Client(n_workers=8)
client

In [ ]:
catalog_computed = dask.compute(tasks)

## Build an Xarray-Datatree from the dictionary of datasets

In [ ]:
dt = DataTree.from_dict(catalog_computed[0])

#### Examine Xarray-Datatree Size

In [ ]:
print(f"Size of data in tree = {dt.nbytes / 1e12 :.2f} TB")

In [ ]:
dt

### Visualize a single dataset with HvPlot

In [ ]:
display(dt["ACCESS-CM2_ssp585"].to_dataset().pr.hvplot("lon", "lat", rasterize=True))